In [1]:
import tweepy
import pandas as pd
import numpy as np
import re

from IPython.display import display
# import matplotlib.pyplot as plt
# import seaborn as sns
# %matplotlib inline

In [2]:
from credentials import *
### credentials file contains my twitter keys and tokens 

def twitter_setup():
    
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN,ACCESS_SECRET)
    
    api = tweepy.API(auth,wait_on_rate_limit = True)
    return api

In [ ]:
## cleaning the tweets by removing links and special links and characters using regrex ##

In [3]:
def clean_tweet(tweet):
    
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [4]:
extractor = twitter_setup()

In [5]:
tweets_list = []

In [ ]:
# streaming the tweets based on the hashtag provided; #MrsMaisel in this particular case

In [6]:
for tweet_info in tweepy.Cursor(extractor.search, 
                                q="#MrsMaisel -filter:retweets", 
#                                 count=5, 
                                tweet_mode="extended", 
                                since="2019-01-22",
                                until="2019-01-23").items(500):
    tweet = clean_tweet(tweet_info.full_text)
    author = tweet_info.user.screen_name
    tweet_time = tweet_info.created_at
    tweet_dict = {'author':author,'time':tweet_time, "tweet_text":tweet}
    
    tweets_list.append(tweet_dict)

In [7]:
df = pd.DataFrame(tweets_list)

In [8]:
df.shape

(49, 3)

In [9]:
df.head()

,author,time,tweet_text
0,JadePagola,2019-01-22 23:59:26,MrsMaisel is one of my favorite shows I highly...
1,luce_taff,2019-01-22 22:11:21,Dr Benjamin Ettenberg is the sweetest fuckin s...
2,Pretty_Dire,2019-01-22 22:10:06,Benjamin MrsMaisel
3,lcamjam,2019-01-22 21:53:40,Just watched the last episode of MrsMaisel sea...
4,EveLaurence,2019-01-22 21:51:48,Midge and Ben at the Art Gallery lol MrsMaisel


In [ ]:
## importing vader

In [10]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

In [ ]:
# define a function that returns the sentiment scores when a string is passed to it 

In [11]:
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score

In [ ]:
## adding a column with the sentiment scores to the dataframe

In [12]:
df["tweet_sentmt"] = df["tweet_text"].apply(sentiment_analyzer_scores)

In [13]:
df.head()

,author,time,tweet_text,tweet_sentmt
0,JadePagola,2019-01-22 23:59:26,MrsMaisel is one of my favorite shows I highly...,"{'neu': 0.608, 'neg': 0.0, 'pos': 0.392, 'comp..."
1,luce_taff,2019-01-22 22:11:21,Dr Benjamin Ettenberg is the sweetest fuckin s...,"{'neu': 1.0, 'neg': 0.0, 'pos': 0.0, 'compound..."
2,Pretty_Dire,2019-01-22 22:10:06,Benjamin MrsMaisel,"{'neu': 1.0, 'neg': 0.0, 'pos': 0.0, 'compound..."
3,lcamjam,2019-01-22 21:53:40,Just watched the last episode of MrsMaisel sea...,"{'neu': 0.493, 'neg': 0.206, 'pos': 0.302, 'co..."
4,EveLaurence,2019-01-22 21:51:48,Midge and Ben at the Art Gallery lol MrsMaisel,"{'neu': 0.741, 'neg': 0.0, 'pos': 0.259, 'comp..."


In [ ]:
## converting the column with the scores from a dictionary to a dataframe

In [14]:
df_sentmt = pd.DataFrame(list(df["tweet_sentmt"]))

In [ ]:
df_sentmt.head()

In [15]:
df = pd.concat([df,df_sentmt], axis = 1)

In [16]:
df.head(10)

,author,time,tweet_text,tweet_sentmt,compound,neg,neu,pos
0,JadePagola,2019-01-22 23:59:26,MrsMaisel is one of my favorite shows I highly...,"{'neu': 0.608, 'neg': 0.0, 'pos': 0.392, 'comp...",0.6997,0.000,0.608,0.392
1,luce_taff,2019-01-22 22:11:21,Dr Benjamin Ettenberg is the sweetest fuckin s...,"{'neu': 1.0, 'neg': 0.0, 'pos': 0.0, 'compound...",0.0000,0.000,1.000,0.000
2,Pretty_Dire,2019-01-22 22:10:06,Benjamin MrsMaisel,"{'neu': 1.0, 'neg': 0.0, 'pos': 0.0, 'compound...",0.0000,0.000,1.000,0.000
3,lcamjam,2019-01-22 21:53:40,Just watched the last episode of MrsMaisel sea...,"{'neu': 0.493, 'neg': 0.206, 'pos': 0.302, 'co...",0.7579,0.206,0.493,0.302
4,EveLaurence,2019-01-22 21:51:48,Midge and Ben at the Art Gallery lol MrsMaisel,"{'neu': 0.741, 'neg': 0.0, 'pos': 0.259, 'comp...",0.4215,0.000,0.741,0.259
5,cheesyearthgirl,2019-01-22 21:48:42,Being pulled in 2 directions Amazon vs Netflix...,"{'neu': 0.842, 'neg': 0.0, 'pos': 0.158, 'comp...",0.5719,0.000,0.842,0.158
6,IntuitLifeCoach,2019-01-22 20:56:29,Sadly community still thinks like this mrsMais...,"{'neu': 0.569, 'neg': 0.228, 'pos': 0.203, 'co...",-0.0772,0.228,0.569,0.203
7,MaiselTV,2019-01-22 20:26:39,She just needed a little vacation MrsMaisel,"{'neu': 1.0, 'neg': 0.0, 'pos': 0.0, 'compound...",0.0000,0.000,1.000,0.000
8,MaiselTV,2019-01-22 20:26:06,Guess Benjamin likes to listen to Ethan s reco...,"{'neu': 0.763, 'neg': 0.0, 'pos': 0.237, 'comp...",0.4215,0.000,0.763,0.237
9,MaiselTV,2019-01-22 20:25:31,Some black and whites might do the trick MrsMa...,"{'neu': 0.87, 'neg': 0.13, 'pos': 0.0, 'compou...",-0.0516,0.130,0.870,0.000


In [ ]:
## average sentiment score for #MrsMaisel

In [17]:
df["compound"].mean()

0.2367

In [18]:
df["neg"].mean()

0.040612244897959185

In [19]:
df["neu"].mean()

0.8171224489795919

In [20]:
df["pos"].mean()

0.14230612244897958